In [1]:
qmd_template = \
"""---
title: '<i class="bi bi-grid-3x3"></i> {title}'
abstract: "{abstract}"
order: {order}
toc: false
listing: 
  template: {ejs_template_path}
  field-required: [variable_name, categories, description]
  filter-ui: true
  sort-ui: false
  categories: numbered
  page-size: 10000
  contents:
    - "{yml_path}"
---

"""

In [2]:
from pathlib import Path
import pandas as pd
import yaml

INPUT_DIR = Path('assets/auto-generated/trial/')
OUTPUT_DIR = Path('spec/trials/')


tables_df = pd.DataFrame(
    yaml.safe_load(open('assets/auto-generated/trial-tables.yml'))
)
tables_df.set_index('name', inplace=True)

EJS_TEMPLATE = Path('assets/templates/trial-table.ejs')

for yml_file in Path(INPUT_DIR).rglob('*.yml'):
    table_name = yml_file.stem.title().replace('-', '')
    table_title = tables_df.loc[table_name, 'label']
    table_abstract = tables_df.loc[table_name, 'description']

    qmd_content = qmd_template.format(
        title=table_title,
        abstract=table_abstract,
        order=tables_df.loc[table_name, 'index'],
        ejs_template_path=EJS_TEMPLATE.relative_to(OUTPUT_DIR, walk_up=True),
        yml_path=yml_file.relative_to(OUTPUT_DIR, walk_up=True)
    )

    qmd_file = OUTPUT_DIR / yml_file.with_suffix('.qmd').name
    qmd_file.parent.mkdir(parents=True, exist_ok=True)
    with open(qmd_file, 'w') as f:
        f.write(qmd_content)

# Write a warning file
with open(OUTPUT_DIR / '_WARNING_', 'w') as f:
    f.write('WARNING: THIS IS AN AUTO-GENERATED DIRECTORY.'
            'DO NOT MANUALLY EDIT ITS CONTENT.\n')